<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


In [2]:
pip install pandoc


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for pandoc: filename=pandoc-2.4-py3-none-any.whl size=34792 sha256=5d346952cccb1d138a67c16d484170ed1f8574ca5bef9c30eee6681857f81fda
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/22/0c/a1/2a5cd1a1bd16658f43498656676770c4fc8a4087eec780df14
Successfully built pandoc
Note: you may need to restart the kernel to use updated packages.


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [7]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [8]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [9]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [10]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [11]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [12]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [13]:
response = requests.get(spacex_url)

Check the content of the response


You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [15]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [16]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [17]:
# Use json_normalize meethod to convert the json result into a dataframeresponse
json_data = response.json()
df = pd.json_normalize(json_data)

Using the dataframe <code>data</code> print the first 5 rows


In [18]:
# Get the head of the dataframe
df.head(5)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [19]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = df[df['cores'].map(len)==1]
data = df[df['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
df['cores'] = df['cores'].map(lambda x : x[0])
df['payloads'] = df['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
df['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
df = df[df['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [20]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [21]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [22]:
# Call getBoosterVersion
getBoosterVersion(data)

the list has now been update 


In [23]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [24]:
# Call getLaunchSite
getLaunchSite(data)

In [25]:


for load in data['payloads']:
    if isinstance(load, str):  # Ensure 'load' is a string
        response = requests.get("https://api.spacexdata.com/v4/payloads/" + load).json()
        PayloadMass.append(response.get('mass_kg', None))  # Handle missing keys gracefully
        Orbit.append(response.get('orbit', None))
    else:
        print(f"Unexpected payload format: {load}")

Unexpected payload format: ['5eb0e4b5b6c3bb0006eeb1e1']
Unexpected payload format: ['5eb0e4b6b6c3bb0006eeb1e2']
Unexpected payload format: ['5eb0e4b7b6c3bb0006eeb1e5']
Unexpected payload format: ['5eb0e4b7b6c3bb0006eeb1e6']
Unexpected payload format: ['5eb0e4b7b6c3bb0006eeb1e7']
Unexpected payload format: ['5eb0e4bab6c3bb0006eeb1ea']
Unexpected payload format: ['5eb0e4bbb6c3bb0006eeb1ed']
Unexpected payload format: ['5eb0e4bbb6c3bb0006eeb1ee']
Unexpected payload format: ['5eb0e4bbb6c3bb0006eeb1ef']
Unexpected payload format: ['5eb0e4bbb6c3bb0006eeb1f0']
Unexpected payload format: ['5eb0e4bbb6c3bb0006eeb1f1']
Unexpected payload format: ['5eb0e4bcb6c3bb0006eeb1f2']
Unexpected payload format: ['5eb0e4bcb6c3bb0006eeb1f3']
Unexpected payload format: ['5eb0e4bcb6c3bb0006eeb1f4']
Unexpected payload format: ['5eb0e4bcb6c3bb0006eeb1f5']
Unexpected payload format: ['5eb0e4bdb6c3bb0006eeb1f6']
Unexpected payload format: ['5eb0e4bdb6c3bb0006eeb1f7']
Unexpected payload format: ['5eb0e4bdb6c3bb0006e

In [26]:
def getCoreData(data):
    for core in data['cores']:  # Iterate over the list of cores
        if isinstance(core, dict) and core.get('core') is not None:  # Ensure core is a dict and has 'core' key
            response = requests.get("https://api.spacexdata.com/v4/cores/" + core['core']).json()
            Block.append(response.get('block', None))  # Handle missing keys gracefully
            ReuseCount.append(response.get('reuse_count', None))
            Status.append(response.get('status', None))


Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [27]:
print(data.keys())


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [28]:
launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date_utc']),  # Use 'date_utc' instead of 'date'
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}


In [29]:
for key, value in launch_dict.items():
    print(f"{key}: {len(value)}")



FlightNumber: 174
Date: 174
BoosterVersion: 174
PayloadMass: 0
Orbit: 0
LaunchSite: 174
Outcome: 0
Flights: 0
GridFins: 0
Reused: 0
Legs: 0
LandingPad: 0
Block: 0
ReusedCount: 0
Serial: 0
Longitude: 174
Latitude: 174


Then, we need to create a Pandas data frame from the dictionary launch_dict.


ValueError: All arrays must be of the same length

In [30]:
# Determine the target length (maximum length among the lists)
target_length = max(len(value) for value in launch_dict.values())

# Align all lists to the target length
for key, value in launch_dict.items():
    if len(value) < target_length:
        # Pad shorter lists with None (or a default value)
        launch_dict[key] += [None] * (target_length - len(value))
    elif len(value) > target_length:
        # Truncate longer lists
        launch_dict[key] = value[:target_length]


In [31]:
# Create a data from launch_dict
launch_df = pd.DataFrame(launch_dict)

# Display the DataFrame
print(launch_df.head())

   FlightNumber                      Date BoosterVersion PayloadMass Orbit  \
0             1  2006-03-24T22:30:00.000Z       Falcon 1        None  None   
1             2  2007-03-21T01:10:00.000Z       Falcon 1        None  None   
2             4  2008-09-28T23:15:00.000Z       Falcon 1        None  None   
3             5  2009-07-13T03:35:00.000Z       Falcon 1        None  None   
4             6  2010-06-04T18:45:00.000Z       Falcon 9        None  None   

        LaunchSite Outcome Flights GridFins Reused  Legs LandingPad Block  \
0  Kwajalein Atoll    None    None     None   None  None       None  None   
1  Kwajalein Atoll    None    None     None   None  None       None  None   
2  Kwajalein Atoll    None    None     None   None  None       None  None   
3  Kwajalein Atoll    None    None     None   None  None       None  None   
4     CCSFS SLC 40    None    None     None   None  None       None  None   

  ReusedCount Serial   Longitude   Latitude  
0        None   None  

Show the summary of the dataframe


In [32]:
# Show the head of the dataframe

print(launch_df.info())  # Provides column data types and non-null counts


print(launch_df.describe())  # Includes count, mean, std, min, max for numeric columns


print(launch_df.head())  # Shows the first 5 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    174 non-null    int64  
 1   Date            174 non-null    object 
 2   BoosterVersion  174 non-null    object 
 3   PayloadMass     0 non-null      object 
 4   Orbit           0 non-null      object 
 5   LaunchSite      174 non-null    object 
 6   Outcome         0 non-null      object 
 7   Flights         0 non-null      object 
 8   GridFins        0 non-null      object 
 9   Reused          0 non-null      object 
 10  Legs            0 non-null      object 
 11  LandingPad      0 non-null      object 
 12  Block           0 non-null      object 
 13  ReusedCount     0 non-null      object 
 14  Serial          0 non-null      object 
 15  Longitude       174 non-null    float64
 16  Latitude        174 non-null    float64
dtypes: float64(2), int64(1), object(14)

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [34]:
# Hint data['BoosterVersion']!='Falcon 1'
#  keep only Falcon 9 launches
data_falcon9 = launch_df[launch_df['BoosterVersion'].str.contains("Falcon 9", na=False)]

# Display the filtered DataFrame
print(data_falcon9.head())  # Preview the first few rows

# Summary of the filtered DataFrame
print(data_falcon9.info())
print(data_falcon9.shape[0])

   FlightNumber                      Date BoosterVersion PayloadMass Orbit  \
4             6  2010-06-04T18:45:00.000Z       Falcon 9        None  None   
5             8  2012-05-22T07:44:00.000Z       Falcon 9        None  None   
6            10  2013-03-01T19:10:00.000Z       Falcon 9        None  None   
7            11  2013-09-29T16:00:00.000Z       Falcon 9        None  None   
8            12  2013-12-03T22:41:00.000Z       Falcon 9        None  None   

     LaunchSite Outcome Flights GridFins Reused  Legs LandingPad Block  \
4  CCSFS SLC 40    None    None     None   None  None       None  None   
5  CCSFS SLC 40    None    None     None   None  None       None  None   
6  CCSFS SLC 40    None    None     None   None  None       None  None   
7   VAFB SLC 4E    None    None     None   None  None       None  None   
8  CCSFS SLC 40    None    None     None   None  None       None  None   

  ReusedCount Serial   Longitude   Latitude  
4        None   None  -80.577366  28.561

Now that we have removed some values we should reset the FlgihtNumber column


In [35]:

# Create a copy of the filtered DataFrame
data_falcon9 = data_falcon9.copy()

# Reset the FlightNumber column
data_falcon9.loc[:, 'FlightNumber'] = list(range(1, data_falcon9.shape[0] + 1))

# Inspect the updated DataFrame
print(data_falcon9.head())


   FlightNumber                      Date BoosterVersion PayloadMass Orbit  \
4             1  2010-06-04T18:45:00.000Z       Falcon 9        None  None   
5             2  2012-05-22T07:44:00.000Z       Falcon 9        None  None   
6             3  2013-03-01T19:10:00.000Z       Falcon 9        None  None   
7             4  2013-09-29T16:00:00.000Z       Falcon 9        None  None   
8             5  2013-12-03T22:41:00.000Z       Falcon 9        None  None   

     LaunchSite Outcome Flights GridFins Reused  Legs LandingPad Block  \
4  CCSFS SLC 40    None    None     None   None  None       None  None   
5  CCSFS SLC 40    None    None     None   None  None       None  None   
6  CCSFS SLC 40    None    None     None   None  None       None  None   
7   VAFB SLC 4E    None    None     None   None  None       None  None   
8  CCSFS SLC 40    None    None     None   None  None       None  None   

  ReusedCount Serial   Longitude   Latitude  
4        None   None  -80.577366  28.561

## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [37]:
data_falcon9.isnull().sum()
nullpads = data_falcon9['LandingPad'].isnull().sum()
print(nullpads)

168


Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [46]:
# Calculate the mean value of PayloadMass column

# Replace the np.nan values with its mean value



# Calculate the mean value of the PayloadMass column, excluding NaN values
mean_payload_mass = data_falcon9['PayloadMass'].mean()

# Replace NaN values in PayloadMass with the calculated mean
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].fillna(mean_payload_mass)

# Inspect the updated column
print(data_falcon9['PayloadMass'])


4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
       ..
169   NaN
170   NaN
171   NaN
172   NaN
173   NaN
Name: PayloadMass, Length: 168, dtype: float64


In [38]:
total_falcon9_launches = data_falcon9.shape[0]

You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


In [39]:
null_landing_pad_rows = data_falcon9[data_falcon9['LandingPad'].isnull()]
print(null_landing_pad_rows)

     FlightNumber                      Date BoosterVersion PayloadMass Orbit  \
4               1  2010-06-04T18:45:00.000Z       Falcon 9        None  None   
5               2  2012-05-22T07:44:00.000Z       Falcon 9        None  None   
6               3  2013-03-01T19:10:00.000Z       Falcon 9        None  None   
7               4  2013-09-29T16:00:00.000Z       Falcon 9        None  None   
8               5  2013-12-03T22:41:00.000Z       Falcon 9        None  None   
..            ...                       ...            ...         ...   ...   
169           164  2022-08-28T02:22:00.000Z       Falcon 9        None  None   
170           165  2022-08-31T05:40:00.000Z       Falcon 9        None  None   
171           166  2022-09-17T01:05:00.000Z       Falcon 9        None  None   
172           167  2022-09-24T23:30:00.000Z       Falcon 9        None  None   
173           168  2022-10-05T16:00:00.000Z       Falcon 9        None  None   

       LaunchSite Outcome Flights GridF

Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright © 2021 IBM Corporation. All rights reserved.
